## Import Dependencies

In [1]:
%%time

import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations
import copy

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pickle

warnings.simplefilter("ignore", category=RuntimeWarning)

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 2.37 s, sys: 262 ms, total: 2.63 s
Wall time: 2.65 s


## Define Case Study Simulator & Functions

In [2]:
def simulator_helper_test_fxns(cs_name, indecies_to_consider, noise_mean, noise_std, normalize, seed):
    """
    Sets the model for calculating y based off of the case study identifier.

    Parameters
    ----------
    cs_name: Class, The name/enumerator associated with the case study being evaluated

    Returns
    -------
    calc_y_fxn: function, the function used for calculation is case study cs_name.name
    """
    #Note: Add your function name from GPBO_Class_fxns.py here
    if cs_name.value == 1:      
        theta_names = ['theta_1', 'theta_2']
        bounds_x_l = [-2]
        bounds_x_u = [2]
        bounds_theta_l = [-2, -2]
        bounds_theta_u = [ 2,  2]
        theta_ref = np.array([1.0, -1.0])     
        calc_y_fxn = calc_cs1_polynomial
        
    elif cs_name.value == 2:                          
        theta_names = ['A_1', 'A_2', 'A_3', 'A_4', 'a_1', 'a_2', 'a_3', 'a_4', 'b_1', 'b_2', 'b_3', 'b_4', 'c_1', 
                       'c_2', 'c_3', 'c_4', 'x0_1', 'x0_2', 'x0_3', 'x0_4', 'x1_1', 'x1_2', 'x1_3', 'x1_4']
        bounds_x_l = [-1.5, -0.5]
        bounds_x_u = [1, 2]
        bounds_theta_l = [-300,-200,-250, 5,-2,-2,-10, -2, -2,-2,5,-2,-20,-20, -10,-1 ,-2,-2,-2, -2,-2,-2,0,-2]
        bounds_theta_u = [-100,  0, -150, 20,2, 2, 0,  2,  2,  2, 15,2, 0,0   , 0,  2, 2,  2, 2, 2 ,2 , 2, 2,2]
        theta_ref = np.array([-200,-100,-170,15,-1,-1,-6.5,0.7,0,0,11,0.6,-10,-10,-6.5,0.7,1,0,-0.5,-1,0,0.5,1.5,1])      
#         theta_ref = np.array([0.5, 0.5, 0.8, 2/3, 0.25, 0.25, 0.35, 0.675, 0.5, 0.5, 0.6, 0.65, 0.5, 0.5, 0.35, 28333/50000, 0.75, 0.5,
#     0.375, 0.25, 0.5, 0.625, 0.75, 0.75])
        calc_y_fxn = calc_muller
        
    else:
        raise ValueError("self.CaseStudyParameters.cs_name.value must exist!")

    return Simulator(indecies_to_consider, 
                     theta_ref,
                     theta_names,
                     bounds_theta_l, 
                     bounds_x_l, 
                     bounds_theta_u, 
                     bounds_x_u, 
                     noise_mean,
                     noise_std,
                     normalize,
                     seed,
                     calc_y_fxn)

## Define Method and DateTime

In [3]:
#Generate Method (except last part)
meth_name = Method_name_enum(5)
method = GPBO_Methods(meth_name)

print("Method Name: ", method.method_name.name)
print("Emulator?: ", method.emulator)
print("Obj Func: ", method.obj.name)
print("Sparse Grid?: ", method.sparse_grid)

#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
# print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M")
DateTime = None ##For Testing

Method Name:  C2
Emulator?:  True
Obj Func:  OBJ
Sparse Grid?:  True


# Case Study GPBO Driver Test

In [4]:
CS_name  = CS_name_enum(1)
print("CS Name: ", CS_name.name)

# indecies_to_consider = list(range(2, 6)) #This is what changes for different subproblems of CS2
indecies_to_consider = list(range(0, 2)) #This is what changes for different subproblems of CS1

ep0 = 1
ep_enum = Ep_enum(1)
sep_fact = 0.8
normalize = False
gen_heat_map_data = True
noise_mean = 0
noise_std = 0.01
# noise_std = 0.0
kernel = Kernel_enum(1)
lenscl = None
outputscl = 1
retrain_GP = 5
reoptimize_obj = 5
bo_iter_tot = 5
bo_run_tot = 3
save_data = True
seed = 1
ei_tol = 1e-6
obj_tol = 1e-4

num_x_data = 5
gen_meth_x = Gen_meth_enum(2) #Note: Has to be the same for validation and sim data
num_theta_data = 20
gen_meth_theta = Gen_meth_enum(1)
num_theta_data_val = 10
gen_meth_theta_val = Gen_meth_enum(1)

CS Name:  CS1


In [5]:
#Set Cs_params and Simulator
cs_name = CS_name.name + "_BO_method_" + meth_name.name + "_sep_fact_" + str(round(sep_fact,2))
cs_params = CaseStudyParameters(cs_name, ep0, sep_fact, normalize, kernel, lenscl, outputscl, retrain_GP, 
                                reoptimize_obj, gen_heat_map_data, bo_iter_tot, bo_run_tot, save_data, DateTime, 
                                seed, ei_tol, obj_tol)

simulator = simulator_helper_test_fxns(CS_name, indecies_to_consider, noise_mean, noise_std, normalize, seed)

#Generate Exp Data
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)

#Generate Sim Data
sim_data = simulator.gen_sim_data(num_theta_data, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, False)
sim_sse_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data, sep_fact, False)

#Generate Validation Data
val_data = simulator.gen_sim_data(num_theta_data_val, num_x_data, gen_meth_theta_val, gen_meth_x, sep_fact, True)
val_sse_data = simulator.sim_data_to_sse_sim_data(method, val_data, exp_data, sep_fact, True)

#Set Ep_Bias
if ep_enum.value == 1:
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, None, None, None, None, None, None)
elif ep_enum.value == 2:
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, bo_iter_tot, None, 0, None, None, None)
elif ep_enum.value == 3:
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, None, 1.5, None, None, None, None)
else:
    ep_bias = Exploration_Bias(None, None, ep_enum, None, None, None, None, None, None, None)

#Initialize Driver
driver = GPBO_Driver(cs_params, method, simulator, exp_data, sim_data, sim_sse_data, val_data, val_sse_data, None, 
                     ep_bias, gen_meth_theta_val)

In [6]:
#Run Bo iterations/restarts
restart_bo_results = driver.run_bo_restarts()

In [7]:
#Open Data File
name_cs_str = "CS1"
fileObj = open("No_Date/Data_Files/" + cs_name + ".pickle", 'rb')
loaded_results = pickle.load(fileObj)
fileObj.close()

#Define the run number and bo iter in question and pull the max bo iters and runs
run_num = 0
bo_iter = 0
runs = loaded_results[run_num].configuration["Number of Workflow Restarts"]
num_sets = loaded_results[run_num].configuration["Max BO Iters"]

#Print Results
print(loaded_results[run_num].configuration)
# print(loaded_results[run_num].list_gp_emulator_class[bo_iter])
# print(len(loaded_results[run_num].list_heat_map_data))
print(loaded_results[run_num].results_df)
print(loaded_results[run_num].results_df["Theta Min Obj Cum."].iloc[-1])
print(loaded_results[run_num].simulator_class.theta_true)
# print(loaded_results[run_num].heat_map_data_dict)
# print(loaded_results[run_num].simulator_class.theta_true_norm)

{'DateTime String': None, 'Method Name Enum Value': 5, 'Case Study Name': 'CS1_BO_method_C2_sep_fact_0.8', 'Number of Parameters': 2, 'Exploration Bias Method Value': 1, 'Separation Factor': 0.8, 'Normalize': False, 'Initial Kernel': <Kernel_enum.MAT_52: 1>, 'Initial Lengthscale': None, 'Initial Outputscale': 1, 'Retrain GP': 5, 'Reoptimize Obj': 5, 'Heat Map Points Generated': True, 'Max BO Iters': 5, 'Number of Workflow Restarts': 3, 'Seed': 1, 'EI Tolerance': 0.0001, 'Obj Improvement Tolerance': 1e-06}
   index Best Error Exploration Bias      Max EI  \
0      0   9.650632                1         0.0   
1      0   9.650632                1  168.808581   

                                Theta Max EI   Min Obj Min Obj Act  \
0  [-1.424906265112171, 0.33997509467786546]   0.00039    0.001695   
1  [0.9885104186344819, -0.9920875149759358]  0.000482    0.003426   

                               Theta Min Obj Min Obj Cum.  \
0  [0.9946435883603465, -0.9935656927586796]     0.001695   